## Hmsc for modeling of species communities

For meaningful ecological analysis, Hmsc model requires a matrix of abundances $Y$, dimensioned $(n_y \times n_s)$, $n_y$ the sampling units (study sites) and $n_s$ the species. The model estimates (regresses) the mean occurrence (abundance or prevalence of the species) against the environmental covariates matrix $X$. One key objective is to relate community-level response to environmental variations (called fixed effects in HMSC context), and the second principal goal is to assess the associative pattern in the residuals (called random effects).

Furthermore, to reflect non-interchangability properties of many filed data and to improve the predictive performance of above simple non-spatial Hmsc model, a spatially explicit model is available for the random effects. It is done by incorporating an additional input with coordinates of the sampling units and making the assumption that a priori the distribution of random effects is not i.i.d., but can reflect the spatial correlation. Alike many other models, used in spatial statistics, this is done by assigning a Gaussian Process (GP) prior for the values of random effects.

However, for large datasets, the full spatial models may become computationally infeasible, since it scales as $n_y^3$. Two alternative approaches are implemented to overcome challenge this in the Hmsc: nearest-neighbour gaussian process (NNGP) and gaussian predictive process (GPP). This facilitates advancement by efficient use of largescale high-resolution ecological datasets.

Additionally, Hmsc is capable of analysis that integrates information like species traits $T$ and phylogenetic relationships $C$ for trait- and pylogeny-level analysis. These extra data adjust the model structure for the fixed effects part of HMSC.

## Clone repo, install dependencies and load packages

We start off with cloning the repository, install missing dependencies, and load packages.

In [ ]:
!git clone https://github.com/aniskhan25/hmsc-hpc.git

Cloning into 'hmsc-hpc'...
remote: Enumerating objects: 707, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 707 (delta 121), reused 125 (delta 65), pack-reused 508
Receiving objects: 100% (707/707), 248.36 KiB | 7.76 MiB/s, done.
Resolving deltas: 100% (449/449), done.


In [ ]:
!pip install ujson pyreadr wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=05a975bb2d739110c160abb883681b56a25ea4732633a5d7ad3b0cee5cba5305
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import os
os.environ['PYTHONPATH'] += f":{os.path.join(os.getcwd(), 'hmsc-hpc')}" # set env variables

!echo $PYTHONPATH

/env/python:/content/hmsc-hpc


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
from platform import python_version

print("Python Version: ", python_version())
print("TF Version: ", tf.__version__)
print("TF Probability Version: ", tfp.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Python Version:  3.10.12
TF Version:  2.14.0
TF Probability Version:  0.22.0
Num GPUs Available:  0


## Load input data and model structure

For this demo notebook, we start with generating a parameter grid based on the aforementioned parameters: the model type (non-spatial and spatial), the number of species $n_s$ and sampling units $n_y$. This grid is used to load pre-saved input data and model structure. For now, the data is for single-chain sampling runs.

In [ ]:
from typing import Iterable, Any
from itertools import product

def grid_parameters(parameters: dict[str, Iterable[Any]]) -> Iterable[dict[str, Any]]:
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))

parameters = {
    'model_type': ['ns', 'fu', 'pg', 'nn', 'ph'] # [non-spatial, full spatial, predictive guassian, nearest neighbor, phylogeny]
    , 'ns': [10, 20, 40, 80, 160, 320, 622] # number of species
    , 'ny': [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25955, 51910, 103820, 207640] # sampling units (site loadings)
    }

nChains = 1

models = {}
for settings in grid_parameters(parameters):
    key = f"{parameters['model_type'].index(settings['model_type'])}{settings['model_type']}_ns{settings['ns']:03d}_ny{settings['ny']:05d}"
    models[key] = settings

print(f"Parameter grid size: {len(models)}")

Parameter grid size: 420


We select an example model from the grid to run.

In [ ]:
model_type = 'ns'
ns = 40 # species
ny = 100 # site loadings

nChains = 1

key = f"{parameters['model_type'].index(model_type)}{model_type}_ns{ns:03d}_ny{ny:05d}"
current_model = models[key]

print(f"Model tested: {current_model}")

Model tested: {'model_type': 'ns', 'ns': 40, 'ny': 100}


The input data and model structure is located on Allas and are downloaded to this notebook session.

In [ ]:
input_filename  = f"init_{key}_chain{nChains:02d}.rds"
output_filename = f"TF_{key}.rds"

input_path  = os.path.join(os.getcwd(),  input_filename)
output_path = os.path.join(os.getcwd(), output_filename)

print(f"Input data and model structure filename: {input_filename}")
print(f"Output posteriors: {output_filename}")

Input data and model structure filename: init_0ns_ns040_ny00100_chain01.rds
Output posteriors: TF_0ns_ns040_ny00100.rds


In [ ]:
import wget

allas_bucket_path = "https://a3s.fi/swift/v1/AUTH_3dd0cc28dd1a45d1a5e119173a48d4f5/2006339-big-spatial-init/"

try:
    wget.download(os.path.join(allas_bucket_path, input_filename))
except Exception as e:
    print(f"Could not download file {input_filename}")
    print(e)

## Run Gibbs sampler

In [ ]:
SAM = 25 # recorded samples from the posterior
THIN = 10 # thinning between recorded samples

In [ ]:
!python hmsc-hpc/hmsc/examples/run_gibbs_sampler.py \
"--input"=$input_path \
"--output"=$output_path \
"--samples"=$SAM \
"--transient"=${SAM*THIN} \
"--thin"=$THIN \
"--verbose"=100 \
"--profile"=1

args=Namespace(samples=25, transient=50, thin=10, input='/content/init_0ns_ns040_ny00100_chain01.rds', output='/content/TF_0ns_ns040_ny00100.rds', verbose=100, tnlib='tf', fse=1, profile=1)
/content
Running TF Gibbs sampler:

Initializing TF graph
retracing
Iterations 2

Completed iterations 2


Computing chain 0
Iterations 300
iteration 300 saving 25
Completed iterations 300

1 chains completed in 0.7 sec

Whole fitting elapsed 0.7


## Optional. List available input data and model structures

Note that the parameter grid contains all possible combinations, but some are missing from the data stored on Allas. These are typically the largest ones, and the reason is either that the initialization object was simply too large to store it with the currently used JSON+RDS approach, or that it is practically infeasible to compute the initialization object due to $n_y^3$ scaling (for GP models `init_1fu_*`)

In [ ]:
import requests

allas_bucket_path = 'https://a3s.fi/swift/v1/AUTH_3dd0cc28dd1a45d1a5e119173a48d4f5/2006339-big-spatial-init/'

files = requests.get(allas_bucket_path).content.decode().split('\n')
for fileName in files:
  print(fileName)

init_0ns_ns040_ny00100_chain01.rds
init_0ns_ns040_ny00200_chain01.rds
init_0ns_ns040_ny00400_chain01.rds
init_0ns_ns040_ny00800_chain01.rds
init_0ns_ns040_ny01600_chain01.rds
init_0ns_ns040_ny03200_chain01.rds
init_0ns_ns040_ny06400_chain01.rds
init_0ns_ns040_ny103820_chain01.rds
init_0ns_ns040_ny12800_chain01.rds
init_0ns_ns040_ny207640_chain01.rds
init_0ns_ns040_ny25955_chain01.rds
init_0ns_ns040_ny51910_chain01.rds
init_0ns_ns160_ny00100_chain01.rds
init_0ns_ns160_ny00200_chain01.rds
init_0ns_ns160_ny00400_chain01.rds
init_0ns_ns160_ny00800_chain01.rds
init_0ns_ns160_ny01600_chain01.rds
init_0ns_ns160_ny03200_chain01.rds
init_0ns_ns160_ny06400_chain01.rds
init_0ns_ns160_ny103820_chain01.rds
init_0ns_ns160_ny12800_chain01.rds
init_0ns_ns160_ny207640_chain01.rds
init_0ns_ns160_ny25955_chain01.rds
init_0ns_ns160_ny51910_chain01.rds
init_0ns_ns622_ny00100_chain01.rds
init_0ns_ns622_ny00200_chain01.rds
init_0ns_ns622_ny00400_chain01.rds
init_0ns_ns622_ny00800_chain01.rds
init_0ns_ns622_n